In [ ]:
import numpy as np
from scipy import signal as signal
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
from scipy.io import wavfile
import scipy.io
import plotly.express as px

init_notebook_mode()

In [ ]:
rawdata=wavfile.read("/home/tfpeng/test8.wav")

In [ ]:
fig=go.Figure()
fig.add_traces([go.Scatter( x=rawdata[1][65555:65555+1024,1],y=rawdata[1][65555:65555+1024,0], mode='markers', name='lines')])
# fig.add_traces([go.Scatter( y=rawdata[1][:,1], mode='lines', name='lines')])
# fig.add_traces([go.Scatter( y=rawdata[1][:,2], mode='lines', name='lines')])
# fig.add_traces([go.Scatter( y=rawdata[1][:,3], mode='lines', name='lines')])
fig.update_layout( autosize=False,
    width=500,
    height=500,)
fig.show()

In [ ]:
def GccPhat(x1, x2,fs, fftlen=None):
    N=len(x1)+len(x2)-1
    x1_fft = np.fft.fft(x1,fftlen)
    x2_fft = np.fft.fft(x2,fftlen)
    G = x1_fft*np.conj(x2_fft)
    Gw = G/(abs(G))
    RR = np.fft.ifft(Gw,16*N)
    # 零频平移
    RR_shift = np.fft.ifftshift(RR)
    index = np.argmax(abs(RR_shift))
    # 计算时延（N为采样点，fs为采样频率）
    delay=(index-16*N/2)/(16*fs)
    return delay

In [ ]:
x1=rawdata[1][65555:65555+512,2]
x2=rawdata[1][65555:65555+512,3]
N=len(x1)
x1_fft = np.fft.fft(x1,512)
x2_fft = np.fft.fft(x2,512)
G = x1_fft*np.conj(x2_fft)
Gw = G/(abs(G))
# Gw=G
RR = np.fft.ifft(Gw,16*512*2)
# 零频平移
RR_shift = np.fft.ifftshift(RR)
index = np.argmax(abs(RR_shift))
# 计算时延（N为采样点，fs为采样频率）
delay_s=(index-16*512)/(16*512*2)
fig=go.Figure()
fig.add_traces([go.Scatter( y=np.abs(RR_shift), mode='lines', name='lines')])
fig.show()

In [ ]:
delay_arr=[]
d=43*0.001
fig=go.Figure()
for i in range(4):
    for j in range(i+1,4):
        x1=rawdata[1][65555:65555+512,i]
        x2=rawdata[1][65555:65555+512,j]
        delay=GccPhat(x1,x2,rawdata[0])
        delay_arr.append(delay)
        print(f"mic {i+1},mic {j+1} delay:{delay}")

In [ ]:
for i in range(3):
    theta=np.arccos(delay_arr[3+i]*340)/d
    print(f"theta = {theta}")

In [26]:
##MVDR
start_point=65555
frame_time=30 #ms
frame_points=int(frame_time*0.001*rawdata[0])
d=0.043
angles=np.arange(-90,90,1)
c=340
# a=np.array([np.exp(-1j*2*np.pi*(-d/c)*np.sin(angles*np.pi/180)),np.ones(len(angles)),np.exp(-1j*2*np.pi*d/c*np.sin((np.pi/3-angles)*np.pi/180))])
a=np.array([np.exp(-1j*2*np.pi*(-d/c)*np.sin((np.pi/2-2*angles)*np.pi/180)),np.ones(len(angles)),np.exp(-1j*2*np.pi*d/c*np.sin((np.pi/6+angles)*np.pi/180))])
frame_num=1
mvdr=[]
for frame in range(frame_num):
    frame_start_index=start_point+frame*frame_points
    x=rawdata[1][frame_start_index:frame_start_index+frame_points,1:]
    RX=np.dot(x.T,x)
    IRX=np.linalg.inv(RX)
    for  i in range(len(angles)):
        mvdr.append(1/(np.dot(a[:,i].T,IRX).dot(a[:,i])))
fig=go.Figure()
fig.add_trace(go.Scatter(x=angles,y=np.abs(mvdr),mode="lines"))
fig.show()


